In [1]:
import pandas as pd 
import numpy as np
import itertools
import os

In [28]:
demand_0 = pd.read_csv('od_demand_distribution_v2.csv',index_col=0)

In [31]:
demand_0= demand_0.rename(columns={'from_master':'from_master_id',
                        'to_master':'to_master_id'})

In [32]:
demand_0.head()

,from_master_id,to_master_id,demand_dist
0,1 Av_0,103 St_0,5.917962
1,1 Av_0,103 St_1,18.316424
2,1 Av_0,103 St_2,3.389157
3,1 Av_0,110 St_0,9.845766
4,1 Av_0,116 St - Columbia University_0,3.609837


In [33]:
demand_0.shape

(110959, 3)

In [5]:
duration = pd.read_csv('remove_one_node_duration_diff.csv.gz',compression='gzip')

In [7]:
del duration['Unnamed: 0']

In [8]:
### if none, means not reachable for 1 hour
duration = duration.fillna(3600)

In [23]:
duration.shape

(215760, 467)

In [34]:
duration_0 = duration.merge(demand_0,on = ['to_master_id','from_master_id'],how='left')

In [35]:
duration_0.shape

(215760, 468)

In [43]:
duration_0['demand_dist']= duration_0['demand_dist'].fillna(0)

In [44]:
duration_0.head()

,to_master_id,from_master_id,Brighton Beach_0,Canal St_1,Canal St_0,Canal St_2,Intervale Av_0,65 St_0,Parkside Av_0,Broad Channel_0,...,59 St - Columbus Circle_0,Pelham Bay Park_0,Flatbush Av - Brooklyn College_0,Jackson Av_0,Nostrand Av_0,Nostrand Av_1,25 Av_0,Neck Rd_0,176 St_0,demand_dist
0,1 Av_0,103 St - Corona Plaza_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.522201
1,1 Av_0,103 St_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.944083
2,1 Av_0,103 St_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.857802
3,1 Av_0,103 St_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1353.69403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.609337
4,1 Av_0,104 St_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.992983


In [45]:
delay = {}
for station in duration_0.columns[2:-1]:
    delay[station]= sum(np.array(duration.loc[:,station]) *\
                        np.array(duration_0.loc[:,'demand_dist']))
    

In [47]:
delay = pd.DataFrame(sorted(delay.items(), key=lambda x: x[1],reverse=True),
                     columns=['station','delay'])

In [48]:
delay.to_csv('delay_multiply_demand_one_node_removal.csv')

In [50]:
critical = delay.head(50)['station'].values

In [51]:
critical

array(['Broadway Jct_0', 'Jackson Hts - Roosevelt Av_0',
       'Kew Gardens - Union Tpke_0', 'Junction Blvd_0',
       '149 St - Grand Concourse_0', '125 St_2', 'Queensboro Plaza_0',
       'Forest Hills - 71 Av_0', '36 St_0', 'Grand Central - 42 St_0',
       '3 Av - 149 St_0', 'Euclid Av_0', '59 St_0', 'Franklin Av_1',
       '39 Av_0', 'Atlantic Av - Barclays Ctr_0',
       '161 St - Yankee Stadium_0', '3 Av - 138 St_0', 'Grant Av_0',
       '36 Av_0', 'Prospect Park_0', 'Parsons Blvd_0', '80 St_0',
       'Church Av_1', 'Broadway_1', '86 St_1', '88 St_0',
       'Hunts Point Av_0', 'E 180 St_0', '167 St_0',
       'Lexington Av/63 St_0', 'Mets - Willets Point_0', 'Rockaway Blvd_0',
       'Times Sq - 42 St_0', '14 St - Union Sq_0', 'President St_0',
       '34 St - Herald Sq_0', '30 Av_0', 'Sterling St_0', '59 St_1',
       'Newkirk Plaza_0', 'Myrtle - Wyckoff Avs_0',
       '59 St - Columbus Circle_0', 'Winthrop St_0', 'Livonia Av_0',
       '168 St - Washington Hts_0', '72 St_2'

In [45]:
critical = [x.replace('/','-') for x in critical]

In [46]:
critical_pairs=[]
for i in itertools.combinations(critical,2):
    critical_pairs.append(i)

In [48]:
critical_pairs[:3]

[('Jackson Hts - Roosevelt Av_0', '149 St - Grand Concourse_0'),
 ('Jackson Hts - Roosevelt Av_0', '125 St_2'),
 ('Jackson Hts - Roosevelt Av_0', 'Broadway Jct_0')]

In [34]:
two_nodes = os.listdir('remove-two-node')

In [35]:
two_nodes = map(lambda x:tuple( x.split(',')), two_nodes)